# Importacoes

In [319]:
import pandas as pd
import duckdb 
import sqlite3
from datetime import datetime
import numpy as np
import regex as re

# Inicio

In [320]:
database_name ="../test_analytics.db"


In [115]:
# def returning_df_from_all_tables(query):
# 	try:

# 		conn = sqlite3.connect(database_name)
# 		cursor = conn.cursor()

# 		cursor.execute (query)
# 		tables = [table[0] for table in cursor.fetchall()]

# 		if not tables:
# 			print ("Not is possible")

# 		union_all_query = " UNION ALL ".join(f"SELECT * FROM \"{table}\"" for table in tables)
# 		#cursor.execute(union_all_query)

# 		# rows = cursor.fetchall()
# 		df = pd.read_sql_query(union_all_query, conn)

# 		return df

# 	except sqlite3.Error as e:
# 		print(f"SQLITE error: {e}")

# 	finally:
# 		if conn:
# 			conn.close()


# query =   """
#   SELECT name FROM sqlite_master
#   WHERE type = 'table' AND  (name LIKE 'escolas%' OR name LIKE '%escolas%');
#   """

# query2 = """
#   SELECT name FROM sqlite_master
#   WHERE type = 'table' AND  (name LIKE 'idadeserie%' OR name LIKE '%idadeserie%');
#   """

# df_escolas = returning_df_from_all_tables(query)

# df_estudantes = returning_df_from_all_tables(query2)

In [116]:
duckdb_database = "../my_database.duckdb"

In [117]:
conn = duckdb.connect(duckdb_database)

In [118]:
# conn.execute("CREATE TABLE IF NOT EXISTS raw_escolas AS SELECT * FROM  df_escolas")

In [119]:
# conn.execute("CREATE TABLE IF NOT EXISTS raw_estudantes AS SELECT * FROM df_estudantes")

In [120]:
escola = conn.sql(
  """
    SELECT *  
    FROM main.raw_escolas re
    WHERE re.fx_etaria IS NOT NULL
    ORDER BY re.codesc;
  """
  )

In [121]:
# endereco = conn.sql(
#   """
#     SELECT DISTINCT
#       re.endereco ,
#       re.bairro ,
#       re.distrito,
#       re.coddist,
#       re.cep,
#       re.latitude ,
#       re.longitude ,
      
#     FROM main.raw_escolas re
#     WHERE re.fx_etaria IS NOT NULL
#     ORDER BY re.codesc;
#   """
#   )

In [122]:
estudantes = conn.sql(

  """
    SELECT * FROM main.raw_estudantes
  
  """

)

In [123]:
df = escola.to_df()

In [124]:
df_estudantes = estudantes.to_df()

# Processing and EDA df escolas

In [125]:
conn.close()

In [321]:
  # # CHECKLIST
  # """
  #   Deleting redundant columns
  #   Renaming the columns
  #   Dropping duplicates
  #   Remove the NaN values from the dataset
  #   Check for some more Transformation

  # """

In [126]:
df_cleaned = df.copy()

In [127]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22894 entries, 0 to 22893
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dre          22894 non-null  object 
 1   codesc       22894 non-null  int64  
 2   tipoesc      22894 non-null  object 
 3   nomesc       22894 non-null  object 
 4   nomescof     18366 non-null  object 
 5   ceu          269 non-null    object 
 6   diretoria    22894 non-null  object 
 7   subpref      22892 non-null  object 
 8   endereco     22894 non-null  object 
 9   numero       22894 non-null  object 
 10  bairro       22894 non-null  object 
 11  cep          22894 non-null  int64  
 12  tel1         18297 non-null  object 
 13  tel2         2770 non-null   object 
 14  fax          2404 non-null   object 
 15  situacao     22894 non-null  object 
 16  coddist      22734 non-null  float64
 17  distrito     22892 non-null  object 
 18  setor        22684 non-null  float64
 19  codi

In [128]:
columns_to_drop = ["ceu", "tel1", "tel2", "fax", "eh", "ato_criacao", "dom_criacao", "dt_ini_func","dt_extincao","dt_autoriza","nome_ant"]

In [129]:
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

In [130]:
columns = df_cleaned.columns

for col in columns:
    if df_cleaned[col].dtype == 'object' or pd.api.types.is_string_dtype(df_cleaned[col]):
        df_cleaned[col] = df_cleaned[col].str.lower().str.strip()

In [131]:
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned = df_cleaned.drop_duplicates(subset='codesc')

In [132]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3531 entries, 0 to 22892
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dre          3531 non-null   object 
 1   codesc       3531 non-null   int64  
 2   tipoesc      3531 non-null   object 
 3   nomesc       3531 non-null   object 
 4   nomescof     3263 non-null   object 
 5   diretoria    3531 non-null   object 
 6   subpref      3531 non-null   object 
 7   endereco     3531 non-null   object 
 8   numero       3531 non-null   object 
 9   bairro       3531 non-null   object 
 10  cep          3531 non-null   int64  
 11  situacao     3531 non-null   object 
 12  coddist      3523 non-null   float64
 13  distrito     3531 non-null   object 
 14  setor        3486 non-null   float64
 15  codinep      3446 non-null   float64
 16  codcie       3412 non-null   float64
 17  fx_etaria    3531 non-null   object 
 18  dt_criacao   2769 non-null   object 
 19  dt_ini_con

In [133]:
# Date Data Constraints Problems
date_columns = ["dt_criacao", "dt_ini_conv","database"]

In [134]:
for col in date_columns:
  df_cleaned[col] = (df_cleaned[col]
                    .str.lower()
                    .str.strip()
                    )
#aux = df_cleaned[date_columns].drop_duplicates().copy()
#print(df_cleaned[date_columns].shape)
#print(df_cleaned[date_columns].drop_duplicates().shape)

In [135]:
date_abv = {
    "jan": '01',
    "fev": '02',
    "mar": '03',
    "abr": '04',
    "mai": '05',
    "jun": '06',
    "jul": '07',
    'ago': '08',
    'set': '09',
    'out': '10',
    'nov': '11',
    'dez': '12',
    '00:00': ''
      }

In [136]:
for col in date_columns:
  df_cleaned[col] = df_cleaned[col].replace(date_abv, regex=True)

for col in date_columns:
  df_cleaned[col] = (df_cleaned[col]
                    .str.lower()
                    .str.strip()
                    )

In [137]:
aux = df_cleaned[date_columns].fillna(np.nan).drop_duplicates().copy()

In [138]:
# Define a function to handle mixed date formats
def parse_dates(date):
    try:
        # Try parsing as DD/MM/YYYY
        return pd.to_datetime(date, format='%d/%m/%Y', errors='raise')
    except:
        try:
            # Try parsing as DD/MM/YY
            return pd.to_datetime(date, format='%d/%m/%y', errors='raise')
        except:
            # Return NaT if parsing fails
            return pd.NaT

In [139]:
aux.tail()

,dt_criacao,dt_ini_conv,database
22888,10/06/2020,NaN,31/12/2020
22889,20/08/20,NaN,31/12/21
22890,18/09/20,NaN,31/12/21
22891,02/10/20,NaN,31/12/21
22892,05/05/21,NaN,31/12/21


In [140]:
aux["dt_criacao"].apply(parse_dates).tail()

22888   2020-06-10
22889   2020-08-20
22890   2020-09-18
22891   2020-10-02
22892   2021-05-05
Name: dt_criacao, dtype: datetime64[ns]

In [141]:
for col in date_columns:
  df_cleaned[col] = df_cleaned[col].apply(parse_dates)

In [142]:
df_cleaned.head()

,dre,codesc,tipoesc,nomesc,nomescof,diretoria,subpref,endereco,numero,bairro,...,setor,codinep,codcie,fx_etaria,dt_criacao,dt_ini_conv,rede,latitude,longitude,database
0,pj,30993,cr.p.conv,vila sulina,vila sulina,pirituba,perus,rua luísa maria rosa,299,vila sulina,...,304.0,35007199.0,7199.0,00 a 03,2020-03-07,NaT,con,-23445107,-46785264,2020-12-31
1,pj,200204,ceci,jaragua,jaragua,pirituba,pirituba,avenida comendador jose de matos,386,vila clarice,...,4202.0,35267181.0,267181.0,zero a 6a e 11m,2004-02-19,2004-12-23,dir,-23463830,-46752228,2010-12-31
10,cs,200205,ceci,krukutu,krukutu,capela do socorro,parelheiros,estrada do curucutu,sn,parelheiros,...,5513.0,35267193.0,267193.0,zero a 6a e 11m,2004-02-19,2004-12-23,dir,-23879692,-46622206,2010-12-31
19,cs,200206,ceci,tenonde pora,tenonde pora,capela do socorro,parelheiros,engenheiro joão lang,153,jardim martini,...,5513.0,35267200.0,267200.0,zero a 6a e11m,2004-02-19,2004-12-23,dir,-23872420,-46650664,2010-12-31
28,sa,300080,esc.part.,jardim da infancia tia lolo,None,diretoria regional de educacao santo amaro,santo amaro,rua itagyba santiago,48,vila alexandria,...,1501.0,35105995.0,NaN,0 a 5 anos,NaT,NaT,None,-23.639.448,-46.669.338,2018-12-30


In [143]:
# df_cleaned = df_cleaned.drop_duplicates()b
# df_cleaned = df_cleaned.drop_duplicates(subset='codesc')

## Cleaning Categorical and Text Columns

In [144]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3531 entries, 0 to 22892
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   dre          3531 non-null   object        
 1   codesc       3531 non-null   int64         
 2   tipoesc      3531 non-null   object        
 3   nomesc       3531 non-null   object        
 4   nomescof     3263 non-null   object        
 5   diretoria    3531 non-null   object        
 6   subpref      3531 non-null   object        
 7   endereco     3531 non-null   object        
 8   numero       3531 non-null   object        
 9   bairro       3531 non-null   object        
 10  cep          3531 non-null   int64         
 11  situacao     3531 non-null   object        
 12  coddist      3523 non-null   float64       
 13  distrito     3531 non-null   object        
 14  setor        3486 non-null   float64       
 15  codinep      3446 non-null   float64       
 16  codcie    

## Diretoria

In [145]:
change_prefix_diretoria = {'diretoria regional de educacao': ''}

In [146]:
aux = df_cleaned.copy()
aux["diretoria"] = aux["diretoria"].replace(change_prefix_diretoria, regex=True)
aux["diretoria"] = aux["diretoria"].str.lower().str.strip()

In [147]:
df_cleaned["diretoria"] = df_cleaned["diretoria"].replace(change_prefix_diretoria, regex=True)
df_cleaned["diretoria"] = df_cleaned["diretoria"].str.lower().str.strip()

## Endereco

In [148]:
def remove_special_characters(text):

  if pd.isna(text):
    return text
  return re.sub(r'[^\w\s]', '', text)

In [149]:
def remove_non_letters(text):
    if pd.isna(text):
        return text
    return re.sub(r'[^a-zA-Z]\s]', '', text)  # Keep only letters a-z or A-Z

In [150]:
def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except (AttributeError, UnicodeEncodeError, UnicodeDecodeError):
        return text  # Return original if encoding fails

In [151]:
def remove_acento_caracteres(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove ~ e outros caracteres especiais
    text = re.sub(r'ç', 'c', text)  # Substitui 'ç' por 'c'
    text = re.sub(r'[áàâãä]', 'a', text)  # Remove acentos em 'a'
    text = re.sub(r'[éèêë]', 'e', text)  # Remove acentos em 'e'
    text = re.sub(r'[íìîï]', 'i', text)  # Remove acentos em 'i'
    text = re.sub(r'[óòôõö]', 'o', text)  # Remove acentos em 'o'
    text = re.sub(r'[úùûü]', 'u', text)  # Remove acentos em 'u'
    return text

In [152]:
replace_columns = {'  ': ''}

In [153]:
df_cleaned["endereco"] = df_cleaned["endereco"].replace(replace_columns, regex=True)
df_cleaned["endereco"] = df_cleaned["endereco"].apply(remove_special_characters)

In [154]:
df_cleaned["bairro"] = df_cleaned["bairro"].apply(remove_special_characters)
df_cleaned["bairro"] = df_cleaned["bairro"].apply(remove_non_letters)
df_cleaned['bairro'] = df_cleaned['bairro'].apply(remove_acento_caracteres)

In [155]:
replace_values = {
  "  ": "",
  
}

In [156]:
df_cleaned["rede"].value_counts()

rede
con    1990
par     889
dir     331
Name: count, dtype: int64

In [157]:
df_escolas_cleaned = df_cleaned.copy()

# EDA df estudantes

In [158]:
df_estudantes.head()

,dre,codesc,tipoesc,nomesc,distrito,setor,ano,rede,modal,descserie,periodo,turno,descturno,sexo,idade,nee,raca,qtde,database
0,BT,191,EMEF,"ALIPIO CORREA NETO, PROF.",VILA SONIA,9404,2016,DIR,ATCOMP,JOGOS EDUCATIVO,DIURNO,3,Tarde,F,12.0,NAO POSSUI,PARDA,2,30/09/2016
1,BT,191,EMEF,"ALIPIO CORREA NETO, PROF.",VILA SONIA,9404,2016,DIR,ATCOMP,JOGOS EDUCATIVO,DIURNO,3,Tarde,F,13.0,NAO POSSUI,NÃO DECLARADA,1,30/09/2016
2,BT,191,EMEF,"ALIPIO CORREA NETO, PROF.",VILA SONIA,9404,2016,DIR,ATCOMP,JOGOS EDUCATIVO,DIURNO,3,Tarde,F,15.0,NAO POSSUI,PARDA,1,30/09/2016
3,BT,191,EMEF,"ALIPIO CORREA NETO, PROF.",VILA SONIA,9404,2016,DIR,ATCOMP,JOGOS EDUCATIVO,DIURNO,3,Tarde,M,11.0,NAO POSSUI,NÃO DECLARADA,1,30/09/2016
4,BT,191,EMEF,"ALIPIO CORREA NETO, PROF.",VILA SONIA,9404,2016,DIR,ATCOMP,JOGOS EDUCATIVO,DIURNO,3,Tarde,M,11.0,NAO POSSUI,PARDA,1,30/09/2016


In [159]:
df_estudantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2982929 entries, 0 to 2982928
Data columns (total 19 columns):
 #   Column     Dtype  
---  ------     -----  
 0   dre        object 
 1   codesc     int64  
 2   tipoesc    object 
 3   nomesc     object 
 4   distrito   object 
 5   setor      int64  
 6   ano        int64  
 7   rede       object 
 8   modal      object 
 9   descserie  object 
 10  periodo    object 
 11  turno      int64  
 12  descturno  object 
 13  sexo       object 
 14  idade      float64
 15  nee        object 
 16  raca       object 
 17  qtde       int64  
 18  database   object 
dtypes: float64(1), int64(5), object(13)
memory usage: 432.4+ MB


In [160]:
import dask.dataframe as dd

## General

In [239]:
df_cleaned = df_estudantes.copy() #dd.from_pandas(df_estudantes.copy(), npartitions=8)
columns = df_cleaned.columns

for col in columns:
    if df_cleaned[col].dtype == 'object' or pd.api.types.is_string_dtype(df_cleaned[col]):
        df_cleaned[col] = df_cleaned[col].str.lower().str.strip()

In [240]:
df_cleaned = df_cleaned.drop_duplicates()

## Date

In [242]:
date_abv = {
    "jan": '01',
    "fev": '02',
    "mar": '03',
    "abr": '04',
    "mai": '05',
    "jun": '06',
    "jul": '07',
    'ago': '08',
    'set': '09',
    'out': '10',
    'nov': '11',
    'dez': '12',
    '00:00': ''
      }

In [243]:
# Define a function to handle mixed date formats
def parse_dates(date):
    try:
        # Try parsing as DD/MM/YYYY
        return pd.to_datetime(date, format='%d/%m/%Y', errors='raise')
    except:
        try:
            # Try parsing as DD/MM/YY
            return pd.to_datetime(date, format='%d/%m/%y', errors='raise')
        except:
            # Return NaT if parsing fails
            return pd.NaT

In [244]:
df_cleaned["database"].unique()

array(['30/09/2016', '28-dez-17', '30/12/2018', '27/12/2019',
       '31/12/2020', '31/12/2021', '30/12/2022 00:00', '25/12/2023 00:00'],
      dtype=object)

In [245]:
# for col in date_columns:
df_cleaned['database'] = df_cleaned['database'].replace(date_abv, regex=True)
#df_cleaned['database'] = df_cleaned['database'].apply(parse_dates).tail()
df_cleaned['database'] = (df_cleaned['database']
                          .str.lower()
                          .str.strip()
                    )

In [246]:
df_cleaned["database"] = pd.to_datetime(df_cleaned["database"], format='mixed', dayfirst=True)

In [248]:
df_cleaned.head()

,dre,codesc,tipoesc,nomesc,distrito,setor,ano,rede,modal,descserie,periodo,turno,descturno,sexo,idade,nee,raca,qtde,database
0,bt,191,emef,"alipio correa neto, prof.",vila sonia,9404,2016,dir,atcomp,jogos educativo,diurno,3,tarde,f,12.0,nao possui,parda,2,2016-09-30
1,bt,191,emef,"alipio correa neto, prof.",vila sonia,9404,2016,dir,atcomp,jogos educativo,diurno,3,tarde,f,13.0,nao possui,não declarada,1,2016-09-30
2,bt,191,emef,"alipio correa neto, prof.",vila sonia,9404,2016,dir,atcomp,jogos educativo,diurno,3,tarde,f,15.0,nao possui,parda,1,2016-09-30
3,bt,191,emef,"alipio correa neto, prof.",vila sonia,9404,2016,dir,atcomp,jogos educativo,diurno,3,tarde,m,11.0,nao possui,não declarada,1,2016-09-30
4,bt,191,emef,"alipio correa neto, prof.",vila sonia,9404,2016,dir,atcomp,jogos educativo,diurno,3,tarde,m,11.0,nao possui,parda,1,2016-09-30


## Text Columns

In [256]:
exclude_cols = ['dre', "tipoesc", "nomesc", "distrito", "setor"]
df_cleaned = df_cleaned.drop(columns=exclude_cols)

In [257]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2797753 entries, 0 to 2982928
Data columns (total 14 columns):
 #   Column     Dtype         
---  ------     -----         
 0   codesc     int64         
 1   ano        int64         
 2   rede       object        
 3   modal      object        
 4   descserie  object        
 5   periodo    object        
 6   turno      int64         
 7   descturno  object        
 8   sexo       object        
 9   idade      float64       
 10  nee        object        
 11  raca       object        
 12  qtde       int64         
 13  database   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(8)
memory usage: 320.2+ MB


In [262]:
df_cleaned.isna().sum()

codesc            0
ano               0
rede           9473
modal         22424
descserie         6
periodo           0
turno             0
descturno         0
sexo            467
idade           449
nee               0
raca         178666
qtde              0
database          0
dtype: int64

In [260]:
df_cleaned["modal"].unique()

array(['atcomp', 'fund', 'rec', 'saai', 'pre', 'eja', 'creche', 'osl',
       'proj', 'mova', 'convee', 'oie', 'espec', 'medio', 'edprof',
       'ensino fundamental de 9 anos', 'educacao infantil',
       'eja escolas ensino fundamental', 'eja cieja',
       'ensino fundamental 9 anos', 'ensino medio', 'tecnico medio',
       'eja escolas educacao especial', 'espec ens medio',
       'ensino medio normal/magisterio', None, 'bercario', 'celp'],
      dtype=object)

In [261]:
new_modals_names = {
  "ensino fundamental de 9 anos": "fund",
  "educacao infantil": "pre",
  "eja escolas ensino fundamental": "eja",
  "eja cieja": 'eja',
  "ensino fundamental 9 anos": "fund",
  "ensino medio": "medio",
  "tecnico medio": "medio",
  "eja escolas educacao especial": "eja",
  "esp ens medio": "espec",
  "ensino medio normal/magisterio": "medio",
  "bercario": "creche"
}

In [263]:
df_cleaned["modal"] = df_cleaned["modal"].replace(new_modals_names, regex=True)

In [274]:
df_cleaned["descturno"].unique()

array(['tarde', 'manhã', 'intermediário', 'noite', 'integral',
       'vespertino', 'manhã\x83â£', 'intermediã\x83â¡rio', 'manhã£',
       'intermediã¡rio'], dtype=object)

In [277]:
df_cleaned["descturno"] = df_cleaned["descturno"].apply(remove_non_letters)
df_cleaned["descturno"] = df_cleaned["descturno"].apply(remove_acento_caracteres)

In [284]:
df_cleaned["descturno"] = df_cleaned["descturno"].replace({"manhaa": "manha", "intermediaario": "intermediario"})

In [285]:
df_cleaned["descturno"].unique()

array(['tarde', 'manha', 'intermediario', 'noite', 'integral',
       'vespertino'], dtype=object)

In [287]:
df_cleaned["nee"].unique()

array(['nao possui', 'def fisica não cadeirante', 'autismo',
       'deficiencia intelectual', 'sindrome de asperger',
       'def fisica cadeirante', 'surdez severa/profunda',
       'deficiencia multipla', 'baixa visao/visao subnormal',
       'surdez leve/moderada', 'altas habilidades/superdotacao',
       'transt desintegrativo infancia', 'cegueira', 'sindrome de rett',
       'surdocegueira', 'deficiencia visual',
       'defic. fisica nao cadeirante',
       'deficiencia multipla nao cadeirante',
       'deficiencia multipla cadeirante',
       'deficiencia fã\x83â\x8dsica cadeirante',
       'baixa visao ou visao subnormal', 'codigo nee desativado',
       'deficiencia fã\x8dsica cadeirante', 'sindrome de down',
       'paralisia cerebral', 'visao monocular'], dtype=object)

In [289]:
df_cleaned["nee"] = df_cleaned["nee"].map(remove_non_letters)
df_cleaned["nee"] = df_cleaned["nee"].map(remove_acento_caracteres)

In [292]:
nee_new_names = {
  "deficiencia faasica cadeirante": "deficiencia fasica cadeirante",
  "surdez severaprofunda": "surdez severa ou profuda",
  "baixa visaovisao subnormal": "baixa visao ou visao subnormal",
  "surdez levemoderada": "surdez leve ou moderada",
  "altas habilidadessuperdotacao": "altas habilidades ou superdotacao",
  "defic fisica nao cadeirante": "def fisica nao cadeirante",
  "deficiencia faasica cadeirante": "deficiencia fisica cadeirante",
  "deficiencia fasica cadeirante": "deficiencia fisica cadeirante"
}

In [294]:
df_cleaned["nee"] = df_cleaned["nee"].replace(nee_new_names)

In [298]:
df_cleaned["raca"].unique()

array(['parda', 'não declarada', 'branca', 'preta', 'indigena', None,
       'amarela', 'nao informada', 'recusou informar', 'recusou informa'],
      dtype=object)

In [300]:
new_names = {
  "não declarada": "nao declarada",
  "nao informada": "nao declarada",
  "recusou informa": "recusou informar"
}

In [301]:
df_cleaned['raca'] = df_cleaned['raca'].replace(new_names)

In [302]:
df_cleaned["raca"].fillna('nao declarada', inplace=True)

In [304]:
df_cleaned["raca"].unique()

array(['parda', 'nao declarada', 'branca', 'preta', 'indigena', 'amarela',
       'recusou informar'], dtype=object)

In [310]:
df_cleaned.drop(columns=["descserie"],inplace=True)

In [311]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2797753 entries, 0 to 2982928
Data columns (total 13 columns):
 #   Column     Dtype         
---  ------     -----         
 0   codesc     int64         
 1   ano        int64         
 2   rede       object        
 3   modal      object        
 4   periodo    object        
 5   turno      int64         
 6   descturno  object        
 7   sexo       object        
 8   idade      float64       
 9   nee        object        
 10  raca       object        
 11  qtde       int64         
 12  database   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(7)
memory usage: 298.8+ MB


In [313]:
df_cleaned = df_cleaned.drop_duplicates()

In [315]:
df_estudantes_cleaned = df_cleaned.copy()

# Loading to Duckdb

In [333]:
df_escolas_cleaned.head()

,dre,codesc,tipoesc,nomesc,nomescof,diretoria,subpref,endereco,numero,bairro,...,setor,codinep,codcie,fx_etaria,dt_criacao,dt_ini_conv,rede,latitude,longitude,database
0,pj,30993,cr.p.conv,vila sulina,vila sulina,pirituba,perus,rua luísa maria rosa,299,vila sulina,...,304.0,35007199.0,7199.0,00 a 03,2020-03-07,NaT,con,-23445107,-46785264,2020-12-31
1,pj,200204,ceci,jaragua,jaragua,pirituba,pirituba,avenida comendadorjose de matos,386,vila clarice,...,4202.0,35267181.0,267181.0,zero a 6a e 11m,2004-02-19,2004-12-23,dir,-23463830,-46752228,2010-12-31
10,cs,200205,ceci,krukutu,krukutu,capela do socorro,parelheiros,estradado curucutu,sn,parelheiros,...,5513.0,35267193.0,267193.0,zero a 6a e 11m,2004-02-19,2004-12-23,dir,-23879692,-46622206,2010-12-31
19,cs,200206,ceci,tenonde pora,tenonde pora,capela do socorro,parelheiros,engenheiro joão lang,153,jardim martini,...,5513.0,35267200.0,267200.0,zero a 6a e11m,2004-02-19,2004-12-23,dir,-23872420,-46650664,2010-12-31
28,sa,300080,esc.part.,jardim da infancia tia lolo,None,santo amaro,santo amaro,rua itagyba santiago,48,vila alexandria,...,1501.0,35105995.0,NaN,0 a 5 anos,NaT,NaT,None,-23.639.448,-46.669.338,2018-12-30


In [337]:
mask = df_escolas_cleaned["codesc"] ==400750

In [338]:
df_escolas_cleaned[mask]

,dre,codesc,tipoesc,nomesc,nomescof,diretoria,subpref,endereco,numero,bairro,...,setor,codinep,codcie,fx_etaria,dt_criacao,dt_ini_conv,rede,latitude,longitude,database
22577,jt,400750,cei indir,cerejeiras,cerejeiras,jacana/tremembe,vila maria/vila guilherme,rua soldado josé higaskino,5,jardim japao,...,8905.0,35474548.0,35474548.0,0a a 3a,2012-06-01,2012-06-01,con,-23510122,-46575400,2012-12-27


In [334]:
df_estudantes_cleaned.head()

,codesc,ano,rede,modal,periodo,turno,descturno,sexo,idade,nee,raca,qtde,database
0,191,2016,dir,atcomp,diurno,3,tarde,f,12.0,nao possui,parda,2,2016-09-30
1,191,2016,dir,atcomp,diurno,3,tarde,f,13.0,nao possui,nao declarada,1,2016-09-30
2,191,2016,dir,atcomp,diurno,3,tarde,f,15.0,nao possui,parda,1,2016-09-30
3,191,2016,dir,atcomp,diurno,3,tarde,m,11.0,nao possui,nao declarada,1,2016-09-30
4,191,2016,dir,atcomp,diurno,3,tarde,m,11.0,nao possui,parda,1,2016-09-30


In [328]:
duckdb_database = "../my_database.duckdb"

In [329]:
conn = duckdb.connect(duckdb_database)

In [330]:
conn.execute("CREATE TABLE IF NOT EXISTS stg_escola AS SELECT * FROM  df_escolas_cleaned")

In [331]:
conn.execute("CREATE TABLE IF NOT EXISTS stg_estudante AS SELECT * FROM  df_estudantes_cleaned")

In [332]:
conn.close()